In [1]:
import numpy as np
import math
import copy
import datetime
import pandas as pd
import json

### Description of add_up_a_decomposition
The function takes as inputs:
1. a decomposition (x_list, BN_matrices_array) output by SER 1, SER 2, GER (x_list is a list, whereas BN_matrices_array is a 2D np array).
2. The dimension of the output TPM row_dim, col_dim

The function then sums up (x_list, BN_matrices_array) and outputs the resulting TPM (row_dim-by-col_dim).

In [2]:
def add_up_a_decomposition(x_list, BN_matrices_array, row_dim, col_dim):
    output_TPM = np.zeros((row_dim, col_dim), dtype=int)
    num_BN_matrices = len(x_list)
    
    for k in range(num_BN_matrices):
        for col in range(col_dim):
            output_TPM[BN_matrices_array[k, col], col] += x_list[k]
    
    return output_TPM

# Implementation of the arXiv Version of GER

## Order of execution: get_col_indices, first_occurrence, column_freq, find_v_list, GERESA, f_score, new_GER.

This function requires two inputs:
1. $P$ is a $2^{n} \times 2^{n}$ numpy array.
2. $z$ is a real number greater than 1.

This function outputs:
1. The length of the decomposition $K$.
2. A list of positive real numbers x_list.
3. A 2D numpy array representing a list of BN matrices A_array.

In [19]:
def new_GER(P, z):
    row_num, col_num = P.shape
    zero_matrix = np.zeros((row_num, col_num), dtype=int)
    
    R = copy.deepcopy(P)
    K = 0
    x_list = []
    BN_list = []
    
    while (not np.array_equal(R, zero_matrix)):
        K += 1
        B = np.min(np.max(R, axis=0))
        v_list = find_v_list(R, B)
        x = 0
        score = -math.inf
        
        for v in v_list:
            temp_A = GERESA(R, v) # temp_A should be a list of length col_num representing a BN matrix.
            
            R_subtract_v_temp_A = copy.deepcopy(R)
            for j in range(col_num):
                R_subtract_v_temp_A[temp_A[j], j] -= v
            
            temp_score = f_score(R_subtract_v_temp_A, z)
            
            if ((temp_score > score) or ((temp_score == score) and (v > x))):
                score = temp_score
                x = v
                A = temp_A
        
        for j in range(col_num):
            R[A[j], j] -= x
        
        x_list.append(x)
        BN_list.append(A)
        
    return K, x_list, np.array(BN_list)

This function requires two inputs:
1. an integer $v$.
2. A 2D numpy array an_array consisting of integers.

The function outputs the column frequency of $v$ in an_array.

In [5]:
def column_freq(v, an_array):
    row_num, col_num = an_array.shape
    col_freq = 0
    
    for j in range(col_num):
        if v in an_array[:, j]:
            col_freq += 1
    
    return col_freq

This function outputs the list of positive entries of $R$ which are not greater than $B$ (upper_bound) and attain the maximum column frequency in $R$.

In [6]:
def find_v_list(R, upper_bound):
    row_num, col_num = R.shape
    classify_entries = [set() for i in range(col_num + 1)]
    
    for i in range(row_num):
        for j in range(col_num):
            if (R[i, j] > 0 and R[i, j] <= upper_bound):
                classify_entries[column_freq(R[i, j], R)].add(R[i, j])
    
    empty_set = set()
    
    for freq in range(col_num, 0, -1):
        if classify_entries[freq] != empty_set:
            return list(classify_entries[freq])

In [7]:
def GERESA(R, v):
    row_num, col_num = R.shape
    A = [0] * col_num
    R_copy = copy.deepcopy(R)
    selected_columns = []
    col_indices_v_R = get_col_indices(v, R)
    col_indices_v_R_complement = set(range(col_num)) - col_indices_v_R
    
    for j in col_indices_v_R:
        A[j] = first_occurrence(v, R[:, j])
        R_copy[A[j], j] = 0
        selected_columns.append(j)
    
    for j in col_indices_v_R_complement:
        # Compute a dictionary with keys in Larger(v, R(:, j)) and values being associated column frequencies.
        dict_KLarger_VColFreq = dict()
        for i in range(row_num):
            if R[i, j] > v:
                dict_KLarger_VColFreq[i] = column_freq(R[i, j] - v, R_copy[:, selected_columns])
        
        # Set A[j] to the right choice from range(row_num).
        A[j] = max(dict_KLarger_VColFreq.keys(), key=dict_KLarger_VColFreq.get)
        
        R_copy[A[j], j] = R[A[j], j] - v
        selected_columns.append(j)
        
    return A

This function requires two inputs:
1. an integer $v$.
2. A 2D numpy array an_array consisting of integers.

The function outputs the set $\text{Col_indices}(v, \text{an_array})$.

In [3]:
def get_col_indices(v, an_array):
    row_num, col_num = an_array.shape
    col_indices = []
    
    for j in range(col_num):
        if v in an_array[:, j]:
            col_indices.append(j)
    
    return set(col_indices)

This function requires two inputs:
1. An integer $v$.
2. A 1D numpy array array_1D such that v is an element of array_1D.

The function outputs the index of the first occurrence of $v$ in array_1D.

In [4]:
def first_occurrence(v, array_1D):
    length = array_1D.size
    
    for i in range(length):
        if v == array_1D[i]:
            return i
    
    return -1

This function requires two inputs:
1. An integer 2D array array_2D.
2. A positive real number $z > 1$.

The function outputs a score.

In [8]:
def f_score(array_2D, z):
    row_num, col_num = array_2D.shape
    score = 0
    checked_positive_entries = set()
    
    for i in range(row_num):
        for j in range(col_num):
            if (array_2D[i, j] > 0) and (not array_2D[i, j] in checked_positive_entries):
                score += z ** column_freq(array_2D[i, j], array_2D)
                checked_positive_entries.add(array_2D[i, j])
    
    return score

# "Truncated Floating-Point Matrix" Experiment

In [18]:
# Custom JSON encoder that handles NumPy types (created by GPT-o3 on poe.com)
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer,)):
            return int(obj)
        elif isinstance(obj, (np.floating,)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

## $8 \times 8$ 5300-TPMs

In [20]:
z_int_list = list(range(1, 5))
z_frac_list = list(range(1, 11))

for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    tTPM_filepath = './8-by-8 (trunc)/dim8_tTPM_' + str(k) + '.npy'
    tTPM = np.load(tTPM_filepath)
    
    for z_int in z_int_list:
        for z_frac in z_frac_list:
            start_time = datetime.datetime.now()
            decom = new_GER(tTPM, z_int + z_frac * 0.1)
            end_time = datetime.datetime.now()
            execution_duration = end_time - start_time
    
            exe_dur_seconds = execution_duration.total_seconds()
            exe_dur_dict = {"execution time": exe_dur_seconds}
    
            # save the data
            decom_filepath = "./raw data (trunc)/8/dim8_decom_m" + str(k) + "_z_" + \
                             str(z_int) + "dot" + str(z_frac) + ".json"
            time_filepath = "./raw data (trunc)/8/dim8_time_m" + str(k) + "_z_" + \
                            str(z_int) + "dot" + str(z_frac) + ".json"
    
            with open(decom_filepath, 'w') as out1:
                json.dump(decom, out1, cls=NumpyEncoder)
        
            with open(time_filepath, 'w') as out2:
                json.dump(exe_dur_dict, out2)
    
        print('Time:', datetime.datetime.now(), '      finished z_int =', z_int)
    
    print()

iteration 0
Time now is: 2025-04-10 18:42:49.354477
Time: 2025-04-10 18:42:57.698701       finished z_int = 1
Time: 2025-04-10 18:43:02.823926       finished z_int = 2
Time: 2025-04-10 18:43:07.958188       finished z_int = 3
Time: 2025-04-10 18:43:12.687825       finished z_int = 4

iteration 1
Time now is: 2025-04-10 18:43:12.688829
Time: 2025-04-10 18:43:25.961284       finished z_int = 1
Time: 2025-04-10 18:43:31.236503       finished z_int = 2
Time: 2025-04-10 18:43:36.317191       finished z_int = 3
Time: 2025-04-10 18:43:41.727022       finished z_int = 4

iteration 2
Time now is: 2025-04-10 18:43:41.727022
Time: 2025-04-10 18:43:53.427619       finished z_int = 1
Time: 2025-04-10 18:43:59.195128       finished z_int = 2
Time: 2025-04-10 18:44:04.947984       finished z_int = 3
Time: 2025-04-10 18:44:11.073419       finished z_int = 4

iteration 3
Time now is: 2025-04-10 18:44:11.073419
Time: 2025-04-10 18:44:24.469203       finished z_int = 1
Time: 2025-04-10 18:44:30.398442   

## $16 \times 16$ 5300-TPMs

In [23]:
z_int_list = list(range(1, 5))
z_frac_list = list(range(1, 11))

for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    tTPM_filepath = './16-by-16 (trunc)/dim16_tTPM_' + str(k) + '.npy'
    tTPM = np.load(tTPM_filepath)
    
    for z_int in z_int_list:
        for z_frac in z_frac_list:
            start_time = datetime.datetime.now()
            decom = new_GER(tTPM, z_int + z_frac * 0.1)
            end_time = datetime.datetime.now()
            execution_duration = end_time - start_time
    
            exe_dur_seconds = execution_duration.total_seconds()
            exe_dur_dict = {"execution time": exe_dur_seconds}
    
            # save the data
            decom_filepath = "./raw data (trunc)/16/dim16_decom_m" + str(k) + "_z_" + \
                             str(z_int) + "dot" + str(z_frac) + ".json"
            time_filepath = "./raw data (trunc)/16/dim16_time_m" + str(k) + "_z_" + \
                            str(z_int) + "dot" + str(z_frac) + ".json"
    
            with open(decom_filepath, 'w') as out1:
                json.dump(decom, out1, cls=NumpyEncoder)
        
            with open(time_filepath, 'w') as out2:
                json.dump(exe_dur_dict, out2)
    
        print('Time:', datetime.datetime.now(), '      finished z_int =', z_int)
    
    print()

iteration 0
Time now is: 2025-04-10 18:49:01.869032
Time: 2025-04-10 18:49:23.905880       finished z_int = 1
Time: 2025-04-10 18:49:42.909080       finished z_int = 2
Time: 2025-04-10 18:50:02.385761       finished z_int = 3
Time: 2025-04-10 18:50:22.416421       finished z_int = 4

iteration 1
Time now is: 2025-04-10 18:50:22.416421
Time: 2025-04-10 18:50:45.047978       finished z_int = 1
Time: 2025-04-10 18:51:07.644012       finished z_int = 2
Time: 2025-04-10 18:51:31.424928       finished z_int = 3
Time: 2025-04-10 18:51:54.953009       finished z_int = 4

iteration 2
Time now is: 2025-04-10 18:51:54.953009
Time: 2025-04-10 18:52:19.789401       finished z_int = 1
Time: 2025-04-10 18:52:39.466828       finished z_int = 2
Time: 2025-04-10 18:53:00.830737       finished z_int = 3
Time: 2025-04-10 18:53:22.617715       finished z_int = 4

iteration 3
Time now is: 2025-04-10 18:53:22.617715
Time: 2025-04-10 18:53:44.068379       finished z_int = 1
Time: 2025-04-10 18:54:05.176192   

## $32 \times 32$ 5300-TPMs

In [24]:
z_int_list = list(range(1, 5))
z_frac_list = list(range(1, 11))

for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    tTPM_filepath = './32-by-32 (trunc)/dim32_tTPM_' + str(k) + '.npy'
    tTPM = np.load(tTPM_filepath)
    
    for z_int in z_int_list:
        for z_frac in z_frac_list:
            start_time = datetime.datetime.now()
            decom = new_GER(tTPM, z_int + z_frac * 0.1)
            end_time = datetime.datetime.now()
            execution_duration = end_time - start_time
    
            exe_dur_seconds = execution_duration.total_seconds()
            exe_dur_dict = {"execution time": exe_dur_seconds}
    
            # save the data
            decom_filepath = "./raw data (trunc)/32/dim32_decom_m" + str(k) + "_z_" + \
                             str(z_int) + "dot" + str(z_frac) + ".json"
            time_filepath = "./raw data (trunc)/32/dim32_time_m" + str(k) + "_z_" + \
                            str(z_int) + "dot" + str(z_frac) + ".json"
    
            with open(decom_filepath, 'w') as out1:
                json.dump(decom, out1, cls=NumpyEncoder)
        
            with open(time_filepath, 'w') as out2:
                json.dump(exe_dur_dict, out2)
    
        print('Time:', datetime.datetime.now(), '      finished z_int =', z_int)
    
    print()

iteration 0
Time now is: 2025-04-10 19:02:20.148345
Time: 2025-04-10 19:04:44.168797       finished z_int = 1
Time: 2025-04-10 19:06:38.299055       finished z_int = 2
Time: 2025-04-10 19:08:07.961687       finished z_int = 3
Time: 2025-04-10 19:09:39.647463       finished z_int = 4

iteration 1
Time now is: 2025-04-10 19:09:39.648804
Time: 2025-04-10 19:11:11.554628       finished z_int = 1
Time: 2025-04-10 19:12:35.534849       finished z_int = 2
Time: 2025-04-10 19:13:57.549111       finished z_int = 3
Time: 2025-04-10 19:15:18.283136       finished z_int = 4

iteration 2
Time now is: 2025-04-10 19:15:18.283136
Time: 2025-04-10 19:16:43.014064       finished z_int = 1
Time: 2025-04-10 19:18:07.382198       finished z_int = 2
Time: 2025-04-10 19:19:31.567081       finished z_int = 3
Time: 2025-04-10 19:20:56.482881       finished z_int = 4

iteration 3
Time now is: 2025-04-10 19:20:56.482881
Time: 2025-04-10 19:22:22.221560       finished z_int = 1
Time: 2025-04-10 19:23:45.439041   

## $64 \times 64$ 5300-TPMs

In [25]:
z_int_list = list(range(1, 5))
z_frac_list = list(range(1, 11))

for k in range(10):
    print('iteration', k)
    print('Time now is:', datetime.datetime.now())
    tTPM_filepath = './64-by-64 (trunc)/dim64_tTPM_' + str(k) + '.npy'
    tTPM = np.load(tTPM_filepath)
    
    for z_int in z_int_list:
        for z_frac in z_frac_list:
            start_time = datetime.datetime.now()
            decom = new_GER(tTPM, z_int + z_frac * 0.1)
            end_time = datetime.datetime.now()
            execution_duration = end_time - start_time
    
            exe_dur_seconds = execution_duration.total_seconds()
            exe_dur_dict = {"execution time": exe_dur_seconds}
    
            # save the data
            decom_filepath = "./raw data (trunc)/64/dim64_decom_m" + str(k) + "_z_" + \
                             str(z_int) + "dot" + str(z_frac) + ".json"
            time_filepath = "./raw data (trunc)/64/dim64_time_m" + str(k) + "_z_" + \
                            str(z_int) + "dot" + str(z_frac) + ".json"
    
            with open(decom_filepath, 'w') as out1:
                json.dump(decom, out1, cls=NumpyEncoder)
        
            with open(time_filepath, 'w') as out2:
                json.dump(exe_dur_dict, out2)
    
        print('Time:', datetime.datetime.now(), '      finished z_int =', z_int)
    
    print()

iteration 0
Time now is: 2025-04-10 20:04:41.698559
Time: 2025-04-10 20:15:48.385317       finished z_int = 1
Time: 2025-04-10 20:26:46.547821       finished z_int = 2
Time: 2025-04-10 20:37:45.195785       finished z_int = 3
Time: 2025-04-10 20:48:44.307204       finished z_int = 4

iteration 1
Time now is: 2025-04-10 20:48:44.307204
Time: 2025-04-10 20:59:26.341032       finished z_int = 1
Time: 2025-04-10 21:10:11.063272       finished z_int = 2
Time: 2025-04-10 21:20:57.332587       finished z_int = 3
Time: 2025-04-10 21:32:54.359612       finished z_int = 4

iteration 2
Time now is: 2025-04-10 21:32:54.359612
Time: 2025-04-10 21:43:39.334722       finished z_int = 1
Time: 2025-04-10 21:56:01.579226       finished z_int = 2
Time: 2025-04-10 22:10:28.573522       finished z_int = 3
Time: 2025-04-10 22:24:48.869210       finished z_int = 4

iteration 3
Time now is: 2025-04-10 22:24:48.869210
Time: 2025-04-10 22:38:37.182089       finished z_int = 1
Time: 2025-04-10 22:52:18.012710   